In [1]:
import requests

In [24]:
import requests

offset_val = 0

# Define the base URL without filter parameters
base_url = "https://data.exactspace.co/exactapi/units/60ae9143e284d016d3559dfb/activities"

all_objects = []  # List to store all fetched objects

while len(all_objects) < 900:  # Continue fetching until desired number is reached
    
    # Construct the URL with the updated offset value
    url = f"{base_url}?filter=%7B%20%20%20%20%20%22where%22%3A%20%7B%20%20%20%20%20%20%20%20%20%22type%22%3A%20%22task%22%20%20%7D%2C%20%20%20%20%20%22order%22%3A%20%22createdOn%20DESC%22%2C%20%20%20%20%20%22limit%22%3A%2010%2C%20%22offset%22%3A{offset_val}%20%7D&access_token=Ziv35SH8DYvWbh0ZL8HMhOCESfQTwajS7j8iHfb1A3pEoSO6P8NraCLmCD1g7iAH"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        objects_fetched = len(data)  # Number of objects fetched in this request
        all_objects.extend(data)      # Add fetched objects to the list
        offset_val += 10
        print("offset success:", offset_val)
    else:
        print("Failed to fetch data. Status code:", response.status_code)
        break

print("Total objects fetched:", len(all_objects))


offset success: 10
offset success: 20
offset success: 30
offset success: 40
offset success: 50
offset success: 60
offset success: 70
offset success: 80
offset success: 90
offset success: 100
offset success: 110
offset success: 120
offset success: 130
offset success: 140
offset success: 150
offset success: 160
offset success: 170
offset success: 180
offset success: 190
offset success: 200
offset success: 210
offset success: 220
offset success: 230
offset success: 240
offset success: 250
offset success: 260
offset success: 270
offset success: 280
offset success: 290
offset success: 300
offset success: 310
offset success: 320
offset success: 330
offset success: 340
offset success: 350
offset success: 360
offset success: 370
offset success: 380
offset success: 390
offset success: 400
offset success: 410
offset success: 420
offset success: 430
offset success: 440
offset success: 450
offset success: 460
offset success: 470
offset success: 480
offset success: 490
offset success: 500
offset su

In [23]:
all_objects[6]

{'type': 'task',
 'voteAcceptCount': 0,
 'voteRejectCount': 0,
 'acceptedUserList': [],
 'rejectedUserList': [],
 'dueDate': '2024-04-23T06:38:57.962Z',
 'assignee': None,
 'source': 'PM',
 'team': 'Maintenance',
 'createdBy': None,
 'createdOn': '2024-04-22T06:38:40.000Z',
 'siteId': '65817c061be4c00007dc7f02',
 'subTaskOf': None,
 'subTasks': [],
 'chats': [],
 'chatOf': None,
 'taskPriority': None,
 'updateHistory': [{'action': 'Pulse created this task',
   'on': '2024-04-22T06:38:40.000Z',
   'by': ''}],
 'id': '6625b8aa3f0e4100065ba5f9',
 'unitsId': '65817c16dcd5de0007bbbf5d',
 'status': 'new',
 'sourceURL': '/rotary-assets/equipment/658192b61be4c00007dc8136?unitId=65817c16dcd5de0007bbbf5d&pm=660fbf49e009d9000790f00b',
 'pmId': '660fbf49e009d9000790f00b',
 'taskGeneratedBy': 'system',
 'recur': True,
 'content': [{'type': 'title', 'value': 'Test 2'}]}

In [60]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # Write tables
    start_row += 1  # Adding some empty rows between subtitles and tables
    for table_data in tables:
        # Set column width
        worksheet.set_column('A:F', 33)
        # Bold and light background color for the first row
        bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
        for j, value in enumerate(table_data[0]):
            worksheet.write(start_row, j, value, bold_format)
        start_row += 1
        for row_data in table_data[1:]:
            for j, value in enumerate(row_data):
                worksheet.write(start_row, j, value)
            start_row += 1
        start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionary
    status_count = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}

    # Count statuses
    for obj in all_objects:
        status = obj.get("status", "--")
        status_count[status] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})

    # Write headers
    statistics_sheet.write('A1', 'Status', bold_format)
    statistics_sheet.write('B1', 'Count', bold_format)

    # Write status counts
    for idx, (status, count) in enumerate(status_count.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj["content"])
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [61]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # Write tables
    start_row += 1  # Adding some empty rows between subtitles and tables
    for table_data in tables:
        # Set column width
        worksheet.set_column('A:F', 33)
        # Bold and light background color for the first row
        bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
        for j, value in enumerate(table_data[0]):
            worksheet.write(start_row, j, value, bold_format)
        start_row += 1
        for row_data in table_data[1:]:
            for j, value in enumerate(row_data):
                worksheet.write(start_row, j, value)
            start_row += 1
        start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}

    # Count statuses
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})

    # Write headers
    statistics_sheet.write('A1', 'Status', bold_format)
    statistics_sheet.write('B1', 'Count (Recommendation)', bold_format)
    statistics_sheet.write('C1', 'Count (Shift Report)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=2):
        statistics_sheet.write(f'C{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [83]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # Write tables
    start_row += 1  # Adding some empty rows between subtitles and tables
    for table_data in tables:
        # Set column width
        worksheet.set_column('A:F', 33)
        # Bold and light background color for the first row
        bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
        for j, value in enumerate(table_data[0]):
            worksheet.write(start_row, j, value, bold_format)
        start_row += 1
        for row_data in table_data[1:]:
            for j, value in enumerate(row_data):
                worksheet.write(start_row, j, value)
            start_row += 1
        start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})
    statistics_sheet.set_column('A:Z', 25)

    # Write headers for Recommendation
    statistics_sheet.write('A1', 'Status(Recommendations)', bold_format)
    statistics_sheet.write('B1', 'Count (Recommendations)', bold_format)
    statistics_sheet.write('C1', 'Action(Recommendations)', bold_format)
    statistics_sheet.write('D1', 'Count(Recommendations)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=2):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

    # Write headers for Shift Report
    statistics_sheet.write('F1', 'Status(Shift Report)', bold_format)
    statistics_sheet.write('G1', 'Count(Shift Report)', bold_format)
    statistics_sheet.write('H1', 'Action(Shift Report)', bold_format)
    statistics_sheet.write('I1', 'Count(Shift Report)', bold_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=2):
        statistics_sheet.write(f'F{idx}', status)
        statistics_sheet.write(f'G{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=2):
        statistics_sheet.write(f'H{idx}', count_type)
        statistics_sheet.write(f'I{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [71]:
import pandas as pd

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # If not update history, write tables
    if update_history:
        # Write tables
        start_row += 1  # Adding some empty rows between subtitles and tables
        for table_data in tables:
            # Set column width
            worksheet.set_column('A:F', 33)
            # Bold and light background color for the first row
            bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
            for j, value in enumerate(table_data[0]):
                worksheet.write(start_row, j, value, bold_format)
            start_row += 1
            for row_data in table_data[1:]:
                for j, value in enumerate(row_data):
                    worksheet.write(start_row, j, value)
                start_row += 1
            start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})
    statistics_sheet.set_column('A:I', 18)

    # Write headers for Recommendation
    statistics_sheet.write('A1', 'Status(Recommendations)', bold_format)
    statistics_sheet.write('B1', 'Count (Recommendations)', bold_format)
    statistics_sheet.write('C1', 'Action(Recommendations)', bold_format)
    statistics_sheet.write('D1', 'Count(Recommendations)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=2):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=2):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

    # Write headers for Shift Report
    statistics_sheet.write('F1', 'Status(Shift Report)', bold_format)
    statistics_sheet.write('G1', 'Count(Shift Report)', bold_format)
    statistics_sheet.write('H1', 'Action(Shift Report)', bold_format)
    statistics_sheet.write('I1', 'Count(Shift Report)', bold_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=2):
        statistics_sheet.write(f'F{idx}', status)
        statistics_sheet.write(f'G{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=2):
        statistics_sheet.write(f'H{idx}', count_type)
        statistics_sheet.write(f'I{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [76]:
import pandas as pd
from datetime import datetime

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # If not update history, write tables
    if update_history:
        # Write tables
        start_row += 1  # Adding some empty rows between subtitles and tables
        for table_data in tables:
            # Set column width
            worksheet.set_column('A:F', 33)
            # Bold and light background color for the first row
            bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
            for j, value in enumerate(table_data[0]):
                worksheet.write(start_row, j, value, bold_format)
            start_row += 1
            for row_data in table_data[1:]:
                for j, value in enumerate(row_data):
                    worksheet.write(start_row, j, value)
                start_row += 1
            start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Get the interval between the first and last object's createdOn times
    first_created_time = None
    last_created_time = None
    for obj in all_objects:
        created_time = datetime.strptime(obj.get("createdOn", ""), "%Y-%m-%dT%H:%M:%S.%fZ")
        if first_created_time is None or created_time < first_created_time:
            first_created_time = created_time
        if last_created_time is None or created_time > last_created_time:
            last_created_time = created_time
    
    interval = (last_created_time - first_created_time).days  # Interval in days

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")

    # Define bold format
    bold_format = workbook.add_format({'bold': True})
    statistics_sheet.set_column('A:G', 25)

    # Write title and interval
    statistics_sheet.write('B1:E1', 'Statistics', bold_format)
    statistics_sheet.write('A3', 'First Created On', bold_format)
    statistics_sheet.write('B3', 'Last Created On', bold_format)
    statistics_sheet.write('C3', 'Interval (Days)', bold_format)

    statistics_sheet.write('A4', first_created_time.strftime("%Y-%m-%d %H:%M:%S"))
    statistics_sheet.write('B4', last_created_time.strftime("%Y-%m-%d %H:%M:%S"))
    statistics_sheet.write('C4', interval)

    # Write headers for Recommendation
    statistics_sheet.write('A7', 'Status(Recommendations)', bold_format)
    statistics_sheet.write('B7', 'Count (Recommendations)', bold_format)
    statistics_sheet.write('C7', 'Action(Recommendations)', bold_format)
    statistics_sheet.write('D7', 'Count(Recommendations)', bold_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=8):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=8):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

    # Write headers for Shift Report
    statistics_sheet.write('A15', 'Status(Shift Report)', bold_format)
    statistics_sheet.write('B15', 'Count(Shift Report)', bold_format)
    statistics_sheet.write('C15', 'Action(Shift Report)', bold_format)
    statistics_sheet.write('D15', 'Count(Shift Report)', bold_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=16):
        statistics_sheet.write(f'A{idx}', status)
        statistics_sheet.write(f'B{idx}', count)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=16):
        statistics_sheet.write(f'C{idx}', count_type)
        statistics_sheet.write(f'D{idx}', count)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [26]:
import pandas as pd
from datetime import datetime

# Function to find or create worksheet based on title
def find_or_create_worksheet(workbook, title):
    for sheet in workbook.worksheets():
        if sheet.get_name().lower() == title.lower():
            return sheet
    return workbook.add_worksheet(title)

# Function to extract title, subtitles, and table data from content
def extract_data(content):
    title = ""
    subtitles = []
    tables = []

    for item in content:
        if item["type"] == "title":
            title = item["value"]
        elif item["type"] == "text":
            subtitles.append(item["value"])
        elif item["type"] == "table":
            tables.append(item["value"])

    return title, subtitles, tables

# Function to write data to Excel
def write_to_excel(title, subtitles, tables, update_history, workbook):
    if "GAP: GSD Recommendations" in title:
        worksheet = find_or_create_worksheet(workbook, "Recommendation")
    elif "GAP Shift" in title:
        worksheet = find_or_create_worksheet(workbook, "Shift Report")
    else:
        return  # Skip writing for other titles

    start_row = worksheet.dim_rowmax or 0  # Get the last used row, or 0 if the sheet is empty

    # Title formatting
    title_format = workbook.add_format({'bold': True, 'font_color': 'white', 'bg_color': '#0070C0', 'align': 'center', 'valign': 'vcenter', 'font_size': 14})
    worksheet.merge_range(f'A{start_row + 1}:F{start_row + 1}', title, title_format)  # Adjusted to start from next row
    start_row += 2  # Move to the next row after the title

    # Subtitles formatting
    subtitle_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#E0E0E0'})
    for subtitle in subtitles:
        worksheet.write(start_row, 0, subtitle, subtitle_format)
        start_row += 1

    # If not update history, write tables
    if update_history:
        # Write tables
        start_row += 1  # Adding some empty rows between subtitles and tables
        for table_data in tables:
            # Set column width
            worksheet.set_column('A:F', 33)
            # Bold and light background color for the first row
            bold_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})
            for j, value in enumerate(table_data[0]):
                worksheet.write(start_row, j, value, bold_format)
            start_row += 1
            for row_data in table_data[1:]:
                for j, value in enumerate(row_data):
                    worksheet.write(start_row, j, value)
                start_row += 1
            start_row += 2  # Adding some empty rows between tables

    # Additional table for updateHistory
    update_history_data = [
        ["Action", "Time"],
        *[(history["action"], history["on"]) for history in update_history]
    ]
    bold_format = workbook.add_format({'bold': True})
    worksheet.write(start_row, 0, "Action Taken in task", bold_format)
    start_row += 1
    for row_data in update_history_data:
        for j, value in enumerate(row_data):
            worksheet.write(start_row, j, value)
        start_row += 1

# Function to write status counts to the "Statistics" sheet
def write_status_counts_to_excel(all_objects, workbook):
    # Initialize status count dictionaries for Recommendation and Shift Report
    status_count_recommendation = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_recommendation = {"Empty": 0, "One comment": 0, "More than one comment": 0}
    status_count_shift_report = {"--": 0, "inprogress": 0, "deferred": 0, "notstarted": 0, "done": 0}
    update_history_count_shift_report = {"Empty": 0, "One comment": 0, "More than one comment": 0}

    # Get the interval between the first and last object's createdOn times
    first_created_time = None
    last_created_time = None
    for obj in all_objects:
        created_time = datetime.strptime(obj.get("createdOn", ""), "%Y-%m-%dT%H:%M:%S.%fZ")
        if first_created_time is None or created_time < first_created_time:
            first_created_time = created_time
        if last_created_time is None or created_time > last_created_time:
            last_created_time = created_time
    
    interval = (last_created_time - first_created_time).days  # Interval in days

    # Count statuses and updateHistory entries
    for obj in all_objects:
        status = obj.get("status", "--")
        title = obj.get("content", [{}])[0].get("value", "")
        update_history = obj.get("updateHistory", [])
        num_comments = len(update_history)

        if "GAP: GSD Recommendations" in title:
            status_count_recommendation[status] += 1
            if num_comments == 0:
                update_history_count_recommendation["Empty"] += 1
            elif num_comments == 1:
                update_history_count_recommendation["One comment"] += 1
            else:
                update_history_count_recommendation["More than one comment"] += 1
        elif "GAP Shift" in title:
            status_count_shift_report[status] += 1
            if num_comments == 0:
                update_history_count_shift_report["Empty"] += 1
            elif num_comments == 1:
                update_history_count_shift_report["One comment"] += 1
            else:
                update_history_count_shift_report["More than one comment"] += 1

    # Create or find the "Statistics" sheet
    statistics_sheet = workbook.add_worksheet("Statistics")
    statistics_sheet.set_column('A:Z', 18)

    # Define bold format and cell formatting
    bold_format = workbook.add_format({'bold': True})
    header_format = workbook.add_format({'bold': True, 'bg_color': '#0070C0', 'font_color': 'white', 'align': 'center', 'valign': 'vcenter'})
    data_format = workbook.add_format({'align': 'center', 'valign': 'vcenter'})
    subtitle_format = workbook.add_format({'bold': True, 'bg_color': '#E0E0E0'})

    # Write title and interval
    statistics_sheet.merge_range('B1:E1', 'Statistics', header_format)
    statistics_sheet.merge_range('A3:A4', 'Created On', subtitle_format)
    statistics_sheet.merge_range('B3:B4', 'Last Created On', subtitle_format)
    statistics_sheet.merge_range('C3:C4', 'Interval (Days)', subtitle_format)

    statistics_sheet.write('A5', first_created_time.strftime("%Y-%m-%d %H:%M:%S"), data_format)
    statistics_sheet.write('B5', last_created_time.strftime("%Y-%m-%d %H:%M:%S"))
    statistics_sheet.write('C5', interval, data_format)

    # Write headers for Recommendation
    statistics_sheet.merge_range('A7:D7', 'Recommendations', header_format)
    statistics_sheet.write('A8', 'Status', header_format)
    statistics_sheet.write('B8', 'Count', header_format)
    statistics_sheet.write('C8', 'Action', header_format)
    statistics_sheet.write('D8', 'Count', header_format)

    # Write status counts for Recommendation
    for idx, (status, count) in enumerate(status_count_recommendation.items(), start=9):
        statistics_sheet.write(f'A{idx}', status, data_format)
        statistics_sheet.write(f'B{idx}', count, data_format)
    for idx, (count_type, count) in enumerate(update_history_count_recommendation.items(), start=9):
        statistics_sheet.write(f'C{idx}', count_type, data_format)
        statistics_sheet.write(f'D{idx}', count, data_format)

    # Write headers for Shift Report
    statistics_sheet.merge_range('F7:I7', 'Shift Report', header_format)
    statistics_sheet.write('F8', 'Status', header_format)
    statistics_sheet.write('G8', 'Count', header_format)
    statistics_sheet.write('H8', 'Action', header_format)
    statistics_sheet.write('I8', 'Count', header_format)

    # Write status counts for Shift Report
    for idx, (status, count) in enumerate(status_count_shift_report.items(), start=9):
        statistics_sheet.write(f'F{idx}', status, data_format)
        statistics_sheet.write(f'G{idx}', count, data_format)
    for idx, (count_type, count) in enumerate(update_history_count_shift_report.items(), start=9):
        statistics_sheet.write(f'H{idx}', count_type, data_format)
        statistics_sheet.write(f'I{idx}', count, data_format)

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    with pd.ExcelWriter('reports_2204.xlsx', engine='xlsxwriter') as writer:
        workbook = writer.book

        for obj in all_objects:
            title, subtitles, tables = extract_data(obj.get("content", []))
            update_history = obj.get("updateHistory", [])  # Get updateHistory data if available
            write_to_excel(title, subtitles, tables, update_history, workbook)

        # Write status counts to the "Statistics" sheet
        write_status_counts_to_excel(all_objects, workbook)

    print("Excel file saved successfully.")

extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [8]:
len(all_objects)

900

In [27]:
import pandas as pd
import json
import requests
from datetime import datetime

tagList = ["GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric"]

def getValues4(tagList, startDate, endDate):
    url =  "https://data.exactspace.co/kairosapi/api/v1/datapoints/query"
    d = {
        "metrics": [
            {
                "tags": {},
                "name": "",
            }
        ],
        "plugins": [],
        "cache_time": 0,
        "start_absolute": startDate,
        "end_absolute": endDate
    }
    dfs = []
    for tag in tagList:
        d['metrics'][0]['name'] = tag
        res = requests.post(url=url, json=d)
        values = json.loads(res.content)
        df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
        df['time'] = pd.to_datetime(df['time'], unit='ms') + pd.Timedelta(hours=5.5)
        df.sort_values(by='time', inplace=True)
        df = df.drop_duplicates(subset=["time", tag])
        if df.shape[0] < 10:
            pass
        else:
            dfs.append(df)
    df = dfs[0]
    for df_ in dfs[1:]:
        df = pd.merge(df, df_, on='time')
    return df

# Function to convert ISO 8601 timestamp to epoch timestamp
def iso_to_epoch(iso_timestamp):
    return int(datetime.fromisoformat(iso_timestamp[:-1]).timestamp())

# Extract recommendation data from JSON and create DataFrame
def extract_recommendations(all_objects):
    recommendations = []
    for obj in all_objects:
        title = obj.get("content", [{}])[0].get("value", "")
        if "GAP: GSD Recommendations" in title:
            recommendation_text = obj.get("content", [{}])[1].get("value", "")  # Assuming the recommendation text is in the second item of the content list
            update_history = obj.get("updateHistory", [])
            update_actions = [entry.get("action", "") for entry in update_history]  # Extract actions from updateHistory
            actions_str = "\n".join(update_actions)  # Concatenate actions into a single string separated by newline
            created_time = obj.get("createdOn")
            due_time = obj.get("dueDate")
            startDate = iso_to_epoch(created_time) * 1000
            endDate = iso_to_epoch(due_time) * 1000
            # Fetch data for the specified time range
            data = getValues4(tagList, startDate, endDate)
            data = data.tail(1)
            data = data.round(4)
            data = data.drop(columns=['time'])
            density = data.to_dict('records')
            result = density[0].get("GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric")
            if result > 1.65:
                density_achieved = "Yes"
            else:
                density_achieved = "No"
            recommendations.append({"ID": obj.get("id", ""), "Recommendation": recommendation_text, "Density Achieved": density_achieved, "Update Actions": actions_str})
    return recommendations

# Extract data from JSON and write to Excel
def extract_and_write_to_excel(all_objects):
    recommendations = extract_recommendations(all_objects)
    if not recommendations:
        print("No recommendations found.")
        return
    
    df = pd.DataFrame(recommendations)
    
    # Define writer
    writer = pd.ExcelWriter("recommendations.xlsx", engine="xlsxwriter")
    df.to_excel(writer, index=False, sheet_name="Recommendations")
    worksheet = writer.sheets["Recommendations"]
    
    # Set column width
    worksheet.set_column('A:Z', 15)  # Set column width for ID
    
    # Increase row height for cells with long text
    for i, text in enumerate(df["Recommendation"], start=1):
        num_lines = text.count("\n") + 1  # Count the number of lines in the text
        if num_lines > 1:
            worksheet.set_row(i, 20 * num_lines)  # Set row height to accommodate the text
        
    writer.save()
    print("Excel file saved successfully.")

# Assuming `all_objects` is defined somewhere
extract_and_write_to_excel(all_objects)


Excel file saved successfully.


In [2]:
import platform
import schedule
import gc
import math
import pandas as pd
import numpy as np
import requests , json
import time
import matplotlib.pyplot as plt
import datetime
import pytz
from pytz import timezone
import platform
import pytz
from datetime import timedelta
# from datetime import datetime
import datetime as datetime
import os

In [ ]:
def run_shiftwise():
    taglist = ['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number','GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric','GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density','GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight','GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']
    taglist1 = ['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number','GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric','GAP_GAP04.PLC04.MLD2_DATA_Anode_Dry_Density','GAP_GAP04.PLC04.MLD2_DATA_Anode_Weight','GAP_GAP04.PLC04.MLD2_DATA_Anode_Height']
    taglist2 = ['GAP_GAP03.PLC03.ACTUAL_FORMULA.FKTP','GAP_GAP04.PLC04.U363_K145_FIT_01_PV','GAP_GAP04.PLC04.level_2_production_reject']
    checklist = ['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE','GAP_GAP04.PLC04.MLD1_DATA_Anode_Number','GAP_GAP04.PLC04.MLD2_DATA_Anode_Number']
    process_tags = [
    "GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS",
    "GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP",
    "GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR",
    "GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF",
    "GAP_GAP01.PLC01._GAPPOS2.PV",
    "GAP_GAP03.PLC03._362_J150_WIT_01.PV", # Mixer weight
    "GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN", # mixer load
    "GAP_GAP03.PLC03._362_J155_WIT_01.PV",  # paste coller weight
    "GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN", # cooler load
    "GAP_GAP04.PLC04.U363_K010_TT_01_PV",
    "GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK",
    "GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres",
    "GAP_GAP04.PLC04.MLD2_DATA_Anode_Vaccum_Pres",
    "GAP_GAP04.PLC04.MLD1_DATA_Anode_Counter_Pres",
    "GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres",
    "GAP_GAP03.PLC03._362_J150B_JT_01.PV", #Mixer pan motor 1 power
    "GAP_GAP03.PLC03._362_J150B_JT_02.PV", #Mixer pan motor 2 power
    "GAP_GAP03.PLC03.U362_J155B_JT_01_PV", #Cooler pan motor 1 power
    "GAP_GAP03.PLC03.U362_J155B_JT_02_PV"] #Cooler pan motor 2 power

    def fetch_data_with_retry(url, payload, max_retries=10, retry_delay=2):
        for attempt in range(max_retries):
            try:
                res = requests.post(url=url, json=payload)
                res.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
                return res
            except requests.exceptions.RequestException as e:
                print(f"Failed to fetch data (attempt {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
        raise RuntimeError("Failed to fetch data after multiple attempts")

    def getValues(tagList):
        # url =config["60ae9143e284d016d3559dfb"]["api"]["query"]
        url='https://data.exactspace.co/kairosapi/api/v1/datapoints/query'
        # print(url)
        d = {
            "metrics": [
                {
                    "tags": {},
                    "name": "",
                    "aggregators": [
                        {
                            "name": "avg",
                            "sampling": {
                                "value": "1",
                                "unit": "minutes"
                            }
                        }
                    ]
                }
            ],
            "plugins": [],
            "cache_time": 0,
            "start_relative": {
                "value": "1",
                "unit": "months"
            }
        }
        finalDF = pd.DataFrame()
        for tag in tagList:
            d['metrics'][0]['name'] = tag
            try:
                res = fetch_data_with_retry(url, d)
                values = json.loads(res.content)
                df = pd.DataFrame(values["queries"][0]["results"][0]['values'], columns=['time', values["queries"][0]["results"][0]['name']])
                finalDF = pd.concat([finalDF, df], axis=1)
            except Exception as e:
                print(f"Failed to fetch data for tag: {tag}. Error: {e}")
                continue
        
        # finalDF.rename(columns=tagList, inplace=True) 
        finalDF = finalDF.loc[:, ~finalDF.columns.duplicated()]
        finalDF.dropna(subset=['time'], inplace=True)
        finalDF['time'] = pd.to_datetime(finalDF['time'], unit='ms') + pd.Timedelta(hours=5.5)
        return finalDF
        
    # print(check_data.head())
    # def status_check():
    #     data= getValues(checklist)
    #     print(data.shape)
    #     data = data[(data['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0)]
    #     data = data[data['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] != data['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift()]
    #     if len(data)>5:
    #         return True
    #     else:
    #         return False
    plant_status = True
    # print(plant_status)

    ##plant_status is equal to true
    if plant_status:
        data = getValues(taglist)
        data1 = getValues(taglist1)
        data2 = getValues(taglist2)
        process_parameters = getValues(process_tags)
        def Current_shift_KPIs(data, data1, data2,process_parameters):
            def convert_values_to_string(dataframes_list):
                return [df.applymap(str) for df in dataframes_list]
            Process_tags_name = ["Time","Green Scrap in formula (%)","Pitch in formula (%)","Fines in formula(%)","Rhodax speed (m/s)","Rhodax Gap (mm)","Mixer weight (kg)","Mixer Rotor Power (kw)","Cooler weight (kg)","Cooler Rotor Power (Kw)","Paste Temperature (deg)","Average Vibration Time (seconds)","Average Vaccum Pressure Mould 1 (mbar) ","Average Vaccum Pressure Mould 2 (mbar)","Counter Pressure Mould 1 (mbar)","Counter Pressure Mould 2 (mbar)","Mixer P1","Mixer P2","Cooler P1","Cooler P2"]

            # Filter and process data
            data = data[(data['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] % 1 == 0)]
            data = data[data['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'] != data['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number'].shift()]

            data1 = data1[(data1['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] % 1 == 0)]
            data1 = data1[data1['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'] != data1['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number'].shift()]
            data.columns = ['Time', 'Mould1_anode_number', 'Mould1_Geometric_density', 'Mould1_Dry_density','Mould1_Anode_weight','Mould1_Anode_Height']
            data1.columns = ['Time', 'Mould2_anode_number', 'Mould2_Geometric_density', 'Mould2_Dry_density','Mould2_Anode_weight','Mould2_Anode_Height']
            data2.columns = ['Time', 'Total Paste', 'Paste Rejection','Total paste production reject']
            
            column_name_M1 = 'Mould1_Anode_weight'
            count_below_1050_M1 = (data[column_name_M1] < 1050).sum()

            column_name_M2 = 'Mould2_Anode_weight'
            count_below_1050_M2 = (data1[column_name_M2] < 1050).sum()

            column_name_Weight_M1 = 'Mould1_Anode_Height'
            count_below_680_M1 = (data[column_name_Weight_M1] < 680).sum()

            column_name_Weight_M2 = 'Mould2_Anode_Height'
            count_below_680_M2 = (data1[column_name_Weight_M2] < 680).sum()
            
            Mould1_anode_Height = data['Mould1_Anode_Height'].mean().round(3)
            Mould2_anode_Height = data1['Mould2_Anode_Height'].mean().round(3)
            Anode_height = (Mould1_anode_Height + Mould2_anode_Height)/2
            Anode_height = Anode_height.round(3)

            percentage_w1 = (count_below_1050_M1 / len(data['Mould1_Anode_weight'])) * 100
            percentage_w2 = (count_below_1050_M2 / len(data1['Mould2_Anode_weight'])) * 100
            percentage_h1 = (count_below_680_M1 / len(data['Mould1_Anode_Height'])) * 100
            percentage_h2 = (count_below_680_M2 / len(data1['Mould2_Anode_Height'])) * 100
            
            Anode1_M1 = (data['Mould1_Geometric_density'] > 1.66 ).sum()
            Anode1_M2 = (data1['Mould2_Geometric_density'] > 1.66 ).sum()
            Anode2_M1 = ((data['Mould1_Geometric_density'] >= 1.65) & (data['Mould1_Geometric_density'] <= 1.66)).sum()
            Anode2_M2 = ((data1['Mould2_Geometric_density'] >= 1.65) & (data1['Mould2_Geometric_density'] <= 1.66)).sum()
            Anode3_M1 = (data['Mould1_Geometric_density'] < 1.65 ).sum()
            Anode3_M2 = (data1['Mould2_Geometric_density'] < 1.65 ).sum()


            # percentage_data = pd.DataFrame({
            #     "Parameters": ["Weight less than 1050(Mould1)", "Weight less than 1050(Mould2)", "Height less than 680(Mould1)", "Height less than 680(Mould2)"],
            #     "No of Anodes": [f' {count_below_1050_M1} anodes', f' {count_below_1050_M2} anodes', f' {count_below_680_M1} anodes', f' {count_below_680_M2}  anodes']
            # })

            quality_metrics_data = pd.DataFrame({
                #"Anode Quality Metrics":[">1.66 in Mould 1",">1.66 in Mould 2","1.65 to 1.66 in Mould 1","1.65 to 1.66 in Mould 2","<1.65 in Mould1 ","<1.65 in Mould2"],
                "Anode Quality Metrics" :['Mould 1 (no of anodes)','Mould 2 (no of anodes)'],
                "Density: >1.66":[f'{Anode1_M1}',f'{Anode1_M2}'],
                "Density: 1.65 to 1.66":[f'{Anode2_M1} ',f'{Anode2_M2}'],
                "Density: < 1.65":[f'{Anode3_M1} ',f'{Anode3_M2} '],
                "Weight : < 1050":[f' {count_below_1050_M1} ', f' {count_below_1050_M2} '],
                "Height : < 680 mm":[f' {count_below_680_M1} ', f' {count_below_680_M2}  '],
            })
            
            
            process_parameters.columns = Process_tags_name
            
            # ist = timezone('Asia/Kolkata')
            # now = datetime.datetime.now(ist)

            # current_hour = now.hour
            # #print(current_hour)
            # # Define the shift based on the current hour
            # if 23 <= current_hour or current_hour < 7:
            #     shift_name = 'Shift B data'
            # elif 7 <= current_hour < 15:
            #     shift_name = 'Shift C data'
            # elif 15 <= current_hour < 23:
            #     shift_name = 'Shift A data'
                
            shift_name = "Shift Report"
            process_parameters = process_parameters[(process_parameters['Rhodax Gap (mm)'] > 16) & (process_parameters['Rhodax Gap (mm)'] < 25)]
            process_parameters = process_parameters[(process_parameters['Paste Temperature (deg)']>160) & (process_parameters['Paste Temperature (deg)']<176)]
            # process_parameters['Mixer weight (kg)']=process_parameters['Mixer weight (kg)']/907.2
            # process_parameters['Cooler weight (kg)']=process_parameters['Cooler weight (kg)']/907.2
            process_parameters['Mixer specific power (Kw/Kg)']=(process_parameters['Mixer P1']+process_parameters['Mixer P2'])/process_parameters['Mixer weight (kg)']
            process_parameters['Cooler specific power (Kw/Kg)']=(process_parameters['Cooler P1']+process_parameters['Cooler P2'])/process_parameters['Cooler weight (kg)']
            # process_parameters['Specific Power (Kw/Ton)'] = ((process_parameters['Mixer P1']+process_parameters['Mixer P2'])+(process_parameters['Cooler P1']+process_parameters['Cooler P2'])) / data2['Total Paste']
            filtered_columns = [col for col in process_parameters.columns if 'P1' not in col and 'P2' not in col and 'Time' not in col]
            average_values = process_parameters[filtered_columns].mean()
            Process_parameters_table = pd.DataFrame({'Process Parameters': average_values.index, shift_name : average_values.values})
            Process_parameters_table = Process_parameters_table.round(3)
            # print(Process_parameters_table)


            # data2.loc[data2['Paste Rejection'] < 0.2, 'Paste Rejection'] = 0
            data = data[data['Mould1_Anode_weight']>1000]
            data1 = data1[data1['Mould2_Anode_weight'] > 1000]
            
            data.reset_index(drop=True, inplace=True)
            data2.reset_index(drop=True, inplace=True)
            data1.reset_index(drop=True, inplace=True)
   
            data2['Total Paste'] = data2['Total Paste'] / 60
            first_value=data2['Total paste production reject'].iloc[0]
            last_value=data2['Total paste production reject'].iloc[-1]
            Total_rejected_paste=(abs(last_value-first_value))/907.2
            # print(Total_rejected_paste)
            Total_rejected_paste=Total_rejected_paste.round(2)
            Total_paste_sum=data2['Total Paste'].sum()
            paste_rejection_percentage=(Total_rejected_paste/Total_paste_sum)*100
            paste_rejection_percentage=paste_rejection_percentage.round(2)
            Total_rejected_paste=str(Total_rejected_paste)+'tons'

            Mould1_anode_weight_mean = data['Mould1_Anode_weight'].mean().round(0)
            Mould2_anode_weight_mean = data1['Mould2_Anode_weight'].mean().round(0)
            Mould1_anode_weight_stdv = data['Mould1_Anode_weight'].std().round(3)
            Mould2_anode_weight_stdv = data1['Mould2_Anode_weight'].std().round(3)
            Mould1_Geometric_mean = data['Mould1_Geometric_density'].mean().round(3)
            Mould2_Geometric_mean = data1['Mould2_Geometric_density'].mean().round(3)
            Mould1_Geometric_stdv = data['Mould1_Geometric_density'].std().round(3)
            Mould2_Geometric_stdv = data1['Mould2_Geometric_density'].std().round(3)
            Mould1_Dry_mean = data['Mould1_Dry_density'].mean().round(3)
            Mould2_Dry_mean = data1['Mould2_Dry_density'].mean().round(3)
            Mould1_Dry_stdv = data['Mould1_Dry_density'].std().round(3)
            Mould2_Dry_stdv = data1['Mould2_Dry_density'].std().round(3)
            Anode_weight = (int(Mould1_anode_weight_mean) +int(Mould2_anode_weight_mean))/2
            Anode_weight = int(Anode_weight)
            combined_mean_Geometric = pd.concat([data['Mould1_Geometric_density'], data1['Mould2_Geometric_density']]).mean().round(3)
            combined_std_dev_Geometric = pd.concat([data['Mould1_Geometric_density'], data1['Mould2_Geometric_density']]).std().round(3)
            combined_std_dev_Dry = pd.concat([data['Mould1_Dry_density'], data1['Mould2_Dry_density']]).std().round(3)
            # print(combined_std_dev_Dry)
            combined_mean_dry = pd.concat([data['Mould1_Dry_density'], data1['Mould2_Dry_density']]).mean().round(3)
            combined_std_dev_weight = pd.concat([data['Mould1_Anode_weight'], data1['Mould2_Anode_weight']]).std().round(3)
            
            table_data1 = {
                'GA Density (g/cm^3)': [combined_mean_Geometric],
                'GA Dry Density(g/cm^3)': [combined_mean_dry],
                'GA Density Stdv': [combined_std_dev_Geometric],
                'GA weight Stdv': [combined_std_dev_weight],
                'Green Paste Rejection (%)': [paste_rejection_percentage]
            }
            table_data = {
                'GA Density (g/cm^3)': [combined_mean_Geometric],
                'GA Dry Density(g/cm^3)': [combined_mean_dry],
                'GA Density Stdv': [combined_std_dev_Geometric],
                'GA Weight':[Anode_weight],
                'GA weight Stdv': [combined_std_dev_weight],
                'Green Paste Rejection (%)': [f' {paste_rejection_percentage}({Total_rejected_paste})'],
                'GA Height (mm)': [Anode_height]
            }

            # Create a DataFrame from the dictionary
            result_table = pd.DataFrame(table_data)
            result_table1 =pd.DataFrame(table_data1)
            # print(result_table)
            
            # Modify the column names accordingly
            result_table = result_table.T
            result_table1 = result_table1.T 
            
            result_table['Targets'] = [ '>=1.650', '>=1.4270', '<=0.005',1050, '< 4','<2.2', 680]
            result_table1['Targets'] = [1.650, 1.4270, 0.005, 4,2.2]
            result_table['Mould 1 Data'] = [Mould1_Geometric_mean,Mould1_Dry_mean,Mould1_Geometric_stdv,Mould1_anode_weight_mean,Mould1_anode_weight_stdv,'-',Mould1_anode_Height]
            result_table['Mould 2 Date'] = [Mould2_Geometric_mean,Mould2_Dry_mean,Mould2_Geometric_stdv,Mould2_anode_weight_mean,Mould2_anode_weight_stdv,'-',Mould2_anode_Height]

            result_table = result_table.rename(columns={0: shift_name})
            result_table1 = result_table1.rename(columns={0: shift_name})

            result_table.reset_index(inplace=True)
            result_table1.reset_index(inplace =True)
            result_table.columns = ['KPIs',shift_name,"Targets",'Mould 1 Data','Mould 2 Data']
            result_table1.columns = ['KPIs',shift_name,"Targets"]
            
                                
            result_table = result_table[['KPIs', 'Targets','Mould 1 Data','Mould 2 Data',shift_name]]
            result_table1 = result_table1[['KPIs', 'Targets', shift_name]]
            result_table = result_table.round(3)
            result_table1 = result_table1.round(3)
            df = result_table1.copy()
            # print(df.head())
            result_table = result_table
            #print(result_table)
            # Determine colors based on whether the target is met
            conditions = ['>=', '>=', '<=', '<=', '<=']
            df['Color'] = ['green' if ((cond == '>=' and current >= target) or
                                            (cond == '<=' and current <= target))
                        else 'salmon' for current, target, cond in zip(df[shift_name], df['Targets'], conditions)]
            display_frames = [result_table]
            display_frames.append(quality_metrics_data)
            display_frames.append(Process_parameters_table)
            # Convert all values in each DataFrame to strings
            display_frames = convert_values_to_string(display_frames)
            return display_frames,df

        def plot_kpi_targets(df):
            DIRECTORY_PATH='kpi_target_img'
            #print(DIRECTORY_PATH)
            ist = timezone('Asia/Kolkata')
            now = datetime.datetime.now(ist)
            current_hour = now.hour

            if 23 <= current_hour or current_hour < 7:
                shift_name = 'Shift B data'
            elif 7 <= current_hour < 15:
                shift_name = 'Shift C data'
            elif 15 <= current_hour < 23:
                shift_name = 'Shift A data'

            fig, ax = plt.subplots(figsize=(15, 9))
            bar_width = 0.35
            index = pd.Index(range(len(df['KPIs'])))

            current_bars = ax.bar(index - bar_width/2, df[shift_name], bar_width, label= shift_name , color=df['Color'])
            target_bars = ax.bar(index + bar_width/2, df['Targets'], bar_width, label='Targets', color='lightgreen')

            for bar in current_bars + target_bars:
                height = bar.get_height()
                ax.annotate('{}'.format(height),
                            xy=(bar.get_x() + bar.get_width() / 2, height),
                            xytext=(0, 3),
                            textcoords="offset points",
                            ha='center', va='bottom')
            ax.legend(handles=[plt.Line2D([0], [0], color='salmon', label='Target Not Achieved'),plt.Line2D([0], [0], color='lightgreen', label='Targets'),plt.Line2D([0], [0], color='green', label=shift_name)])
            ax.set_xlabel('KPIs')
            ax.set_ylabel('Values')
            ax.set_title( shift_name + ' vs Targets')
            ax.set_xticks(index)
            ax.set_xticklabels(df['KPIs'], rotation=45, ha='right')
            #ax.legend()
            print("Saving images to:", DIRECTORY_PATH)
            if not os.path.exists(DIRECTORY_PATH):
               os.makedirs(DIRECTORY_PATH)
            image_path = os.path.join(DIRECTORY_PATH, 'plot_image.png')
            try:
                plt.savefig(image_path)
                plt.close()  # Close the plot to free up memory
                print("Image saved at:", image_path)
            except Exception as e:
                print("Error saving image:", e) 
            return image_path
        
        dataframes,df = Current_shift_KPIs(data, data1, data2,process_parameters)
        df = df.round(3)
        image_path = plot_kpi_targets(df)
        print(image_path)
        
        def uploadRefernceData(fileName):
            #print(fileName)
            #print(type(fileName))
            str_fileName = str(fileName)
            
            path = ""
            files = {'upload_file': open(str(path+str_fileName),'rb')}
            #print(files)
            url='https://data.exactspace.co/exactapi/attachments/tasks/upload'
            response = requests.post(url, files=files)
            print ("uploading")
            #print (url)
            #print ("+"*20)

            if(response.status_code==200):
                status ="success"
                data = response.content
                # Parse the JSON data
                parsed_data = json.loads(data)
                # Access the "name" from the parsed JSON data
                name = parsed_data['result']['files']['upload_file'][0]['name']
                return "https://data.exactspace.co/exactapi/attachments/tasks/download/"+name

                
            else:
                status= (str(response.status_code) + str(response.content))
                print (response.status_code, response.content)

            return status
        image_link = uploadRefernceData(image_path)
        print("this is uploaded image link:", image_link)
        current_date_time = datetime.datetime.now() + pd.Timedelta(hours=5.5)
        task_creation_time = current_date_time.strftime("%Y-%m-%d %H:%M:%S")
        json_body = {
            "type": "task",
                "voteAcceptCount": 0,
                "voteRejectCount": 0,
                "acceptedUserList": [],
                "rejectedUserList": [],
                "dueDate": "2023-11-21T16:00:00.391Z",
                "assignee": "6149b9acf1902b2b7aecf9b1",#anisha
                # "assignee":'632d3bd36d161904360db797', #intern
                "source": "Anode Forming",
                "team": "GAP Shift Report",
                "createdBy": "5f491bb942ba5c3f7a474d15",
                "createdOn":  "2023-11-21T14:49:03.633Z",
                "lastUpdatedOn":"2023-11-21T14:49:03.633Z",
                "siteId": "60ae7260e284d016d3559d09",
                "unique":"KPIs",
                "subTasks": [],
                "chats": [],
                "taskPriority": "high",
                "updateHistory":[{
                    "action":"This task is created by Pulse.",
                     "by": "",
                     "on": task_creation_time
                }],
                "unitsId": "60ae9143e284d016d3559dfb",
                "collaborators": [
                    "632d3bd36d161904360db797" #intern
                    '5c591d697dc9e324ee08a456', 
                    '61431baf1c46e3435ff50ac7', #sayan sir 
                    '5f491bb942ba5c3f7a474d15', 
                    '6149b95af1902b2b7aecf9ac', # anurag sir
                    '6542666c3a469b00079087e8', # aswini mam
                    '6542668db3e4990006874e9b', # dibendu sir
                    '6561adf4bd95b70007098856', #satyanand sir
                    '6561ae15c417de00073b62f7', #saurab sir
                    '6561ae33c417de00073b62f8', #akhilesh sir
                    '6565801507703c0007d79240', #arunk sir
                    '65a6356760b4b400074bbf41', #jitendra sir
                    '65a6379c092148000766155b', #atul sir
                    '65f180008afd390007b693ee', #sushil sir
                    '65f180390ef6370007eb399a' #rahul sir
                ],
                "status": "inprogress",
                "content": [
                    {
                        "type": "title",
                        "value": ""
                    }
                ],
                "taskGeneratedBy": "system",
                "incidentId": "",
                "category": "",
                "sourceURL": "",
                "notifyEmailIds": [
                ],
                "chat": [],
                "taskDescription": "<p><img src=\"{}\"></p>".format(image_link),
                "triggerTimePeriod": "days",
                "viewedUsers": [],
                "completedBy": "",
                "equipmentIds": [],
                "mentions": [],
                "systems": []
            }
        
        def update_json(json_body, dataframes):
            # Get the current time in IST
            ist = timezone('Asia/Kolkata')
            now = datetime.datetime.now(ist)
            due_date = now + timedelta(hours=5)
            json_body["dueDate"] = due_date.replace(tzinfo=None).isoformat()  # Remove the timezone info
            json_body["createdOn"] = now.replace(tzinfo=None).isoformat()  # Remove the timezone info
            current_hour = now.hour
            #print(current_hour)
            current_date = datetime.datetime.now().strftime("%d-%m-%Y")

            # Calculate the previous day's date
            previous_day_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%d-%m-%Y")

            # Your existing logic for determining the title
            global title
            if 23 <= current_hour or current_hour < 7:
                title = f'GAP Shift B KPI report ({current_date})'
            elif 7 <= current_hour < 15:
                title = f'GAP Shift C KPI report ({previous_day_date})'
            elif 15 <= current_hour < 23:
                title = f'GAP Shift A KPI report ({current_date})'
        
            json_body["content"][0]["value"] = title
            for dataframe in dataframes:
                new_table_data = [dataframe.columns.tolist()] + dataframe.values.tolist()
                json_body["content"].append({"type": "table", "value": new_table_data})
            return json_body
        
        def create_task_link(task_id):
            try:
                return 'https://data.exactspace.co/pulse-master/my-tasks/'+ task_id
            except Exception as e:
                print("error in creating the link")
            return
        
        def sendkpiEmail(taskdetails):
            
            # print(taskdetails)
    
            n = 1
            unitName, SiteName, CustomerName = 'GAP', 'GAP Mahan', 'Green Anode Plant, Mahan'
            
            emailTemplate = os.path.join(os.getcwd(), 'kpiEmailTemplate.html')
            
            with open(emailTemplate, 'r') as f:
                s = f.read()

            # if PUBLIC_DATACENTER_URL != 'NA':
            #     logoLink = 'img src="{}pulse-files/email-logos/logo.png"'.format(PUBLIC_DATACENTER_URL)
            #     s = s.replace('img src="#"', logoLink)
            # else:
            logoLink ='https://data.exactspace.co/attachments/mail/download/logo.png'
            s = s.replace('img src="#"', 'img src="{}"'.format(logoLink))

                
            s = s.replace("""<a style="color: #fff; text-decoration:none;" href="#" id = 'task_link'>More Details</a>""", 
                          """<a style="color: #fff; text-decoration:none;" href="{}" id = 'task_link'>More Details</a>""".format(taskdetails.get("task_link")))

            # print(taskdetails.get("link"))
            
            s = s.replace('UnitName', unitName)
            s = s.replace('SiteName', SiteName)
            s = s.replace('CustomerName', CustomerName)
            
            devTable = ''
            devTable = '<tbody id="devList">'
            try:
                devTable += '''
                    <tr>
                        <td align="center" width="40" style="border-bottom: solid 1px #CACACA;">{}</td>
                        <td align="left" style="font-size: 13px; border-bottom: solid 1px #CACACA;">{}</td>
                    </tr>
                '''.format(n, taskdetails.get("desc", ''))
            except Exception as e:
                print('Error in creating a table of alert', e)
                return

            s = s.replace('<tbody id="devList">', devTable)

            with open(os.path.join(os.getcwd(), 'almEmailTemp.html'), 'wb') as f:
                f.write(s.encode('utf-8'))

            with open(os.path.join(os.getcwd(), 'almEmailTemp.html'), 'r') as f:
                msg_body = f.read()

            try:
                url = config['api']['meta'].replace('exactapi', 'mail/send-mail')
                # print(url)
                payload = json.dumps({
                    'from': 'shashank.r@exactspace.co',
                    'to': ['sayan.dey@adityabirla.com',
                            'anurag.gaurav@adityabirla.com',
                            'anisha.jonnalagadda@adityabirla.com',
                            'aswini.mishra@adityabirla.com',
                            'dibyendu.g@adityabirla.com',
                            'satanand.vaishya@adityabirla.com',
                            'sourabh.chourasiya@adityabirla.com',
                            'akhilesh.prasad@adityabirla.com',
                            'arunk.panday@adityabirla.com',
                            'jitendra.prajapati@adityabirla.com',
                            'atul.a.yadav@adityabirla.com',
                            'sushil.yadav@adityabirla.com',
                            'rahula.kumar@adityabirla.com'
                    ],
                    # 'to':['shashank.r@exactspace.co'],
                    'html': msg_body,
                    'cc': ['shashank.r@exactspace.co','sairam.g@exactspace.co','ashlin.f@exactspace.co','nikhil.s@exactpsace.co'],
                    'subject': 'KPI Parameters Report',
                    'body': msg_body
                 })
                headers = {'Content-Type': 'application/json'}
                response = requests.post(url, data=payload, headers=headers)
                # print(response)

                if response.text == 'Success':
                    return 'Success'
                else:
                    print('Error in sending mail', response.status_code)
                    return 'Fail'

            except Exception as e:
                print('Error in sending mail', e)
                return 'Fail'
            finally:
                s=None
                msg_body=None
                
        updates_in_json = update_json(json_body,dataframes)
        #print(updates_in_json)
        json_data = json.dumps(updates_in_json)
        print(json_data)

        # Set the headers to indicate that you are sending JSON data
        headers = {"Content-Type": "application/json"}
        #print(data)
        post_url='https://data.exactspace.co/exactapi/activities'
        # Make the POST request
        response = requests.post(post_url, data=json_data, headers=headers)
        # Check the response
        if response.status_code == 200:
            print("Task Create request was successful")
            response_data = response.json()
            # print(response_data)
            global last_task_id
            last_task_id = response_data.get('id')
            # print(last_task_id)
            task_link = create_task_link(last_task_id)
            # print(task_link)
            
            sendkpiEmail({"desc":title,"task_link":task_link})

        else:
            print("Task Create request failed with status code:", response.status_code)
    else:
        print("Plant is Shutdown")
    gc.collect()
     

In [3]:
taglist = ['GAP_GAP04.PLC04.MLD1_DATA_Anode_Number','GAP_GAP04.PLC04.MLD1_DATA_Anode_Geometric','GAP_GAP04.PLC04.MLD1_DATA_Anode_Dry_Density','GAP_GAP04.PLC04.MLD1_DATA_Anode_Weight','GAP_GAP04.PLC04.MLD1_DATA_Anode_Height']
taglist1 = ['GAP_GAP04.PLC04.MLD2_DATA_Anode_Number','GAP_GAP04.PLC04.MLD2_DATA_Anode_Geometric','GAP_GAP04.PLC04.MLD2_DATA_Anode_Dry_Density','GAP_GAP04.PLC04.MLD2_DATA_Anode_Weight','GAP_GAP04.PLC04.MLD2_DATA_Anode_Height']
taglist2 = ['GAP_GAP03.PLC03.ACTUAL_FORMULA.FKTP','GAP_GAP04.PLC04.U363_K145_FIT_01_PV','GAP_GAP04.PLC04.level_2_production_reject']
checklist = ['GAP_GAP03.PLC03.SCHENCK2_FEED_RATE','GAP_GAP04.PLC04.MLD1_DATA_Anode_Number','GAP_GAP04.PLC04.MLD2_DATA_Anode_Number']
process_tags = [
"GAP_GAP03.PLC03.ACTUAL_FORMULA.KGS",
"GAP_GAP03.PLC03.ACTUAL_FORMULA.KLP",
"GAP_GAP03.PLC03.ACTUAL_FORMULA.KFR",
"GAP_GAP01.PLC01.U362_E020_MVF_01_ACTRL_AUTOSPEEDREF",
"GAP_GAP01.PLC01._GAPPOS2.PV",
"GAP_GAP03.PLC03._362_J150_WIT_01.PV", # Mixer weight
"GAP_GAP03.PLC03.J362_J150_JT_01_PW01_IN", # mixer load
"GAP_GAP03.PLC03._362_J155_WIT_01.PV",  # paste coller weight
"GAP_GAP03.PLC03.J362_J155_JT_01_PW01_IN", # cooler load
"GAP_GAP04.PLC04.U363_K010_TT_01_PV",
"GAP_GAP04.PLC04.K363_K040A_MVF_01_VTK",
"GAP_GAP04.PLC04.MLD1_DATA_Anode_Vaccum_Pres",
"GAP_GAP04.PLC04.MLD2_DATA_Anode_Vaccum_Pres",
"GAP_GAP04.PLC04.MLD1_DATA_Anode_Counter_Pres",
"GAP_GAP04.PLC04.MLD2_DATA_Anode_Counter_Pres",
"GAP_GAP03.PLC03._362_J150B_JT_01.PV", #Mixer pan motor 1 power
"GAP_GAP03.PLC03._362_J150B_JT_02.PV", #Mixer pan motor 2 power
"GAP_GAP03.PLC03.U362_J155B_JT_01_PV", #Cooler pan motor 1 power
"GAP_GAP03.PLC03.U362_J155B_JT_02_PV"] #Cooler pan motor 2 power

In [28]:
import pandas as pd

# Load the Excel file
excel_file = 'recommendations.xlsx'
df = pd.read_excel(excel_file)

# Define the phrase to remove
phrase_to_remove = "This task is created by Pulse."

# Remove the phrase from the specified column
df['Update Actions'] = df['Update Actions'].str.replace(phrase_to_remove, '')

# Save the modified DataFrame back to Excel
output_file = 'final_report.xlsx'
df.to_excel(output_file, index=False)

print("Data cleaned and saved successfully.")


Data cleaned and saved successfully.


c:\Users\vikra\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
